In [ ]:
'''
This takes the results from network_processing.ipynb - all Enriched Genes based on DEGs, alphas and Enrichment Scores
It filters the original VST normalized transcriptomic data to only those DEGs.
This data will be run through WGCNA for module detection.

'''

In [1]:
import pandas as pd
import numpy as np

In [5]:
filtered_gestalt = pd.read_csv('gestalt_dyngenie.csv')
read_counts = pd.read_csv('rna_vst_proc.csv')
kegg_ids = pd.read_csv('../user_ko.txt',sep='\t')

/var/folders/v1/ymptnwwx3x1b7_mw0nns09wr0000gn/T/ipykernel_28793/331907754.py:2: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160) have mixed types. Specify dtype option on import or set low_memory=False.
  read_counts = pd.read_csv('rna_vst_proc.csv')


In [31]:
#clean kegg_id to drop t1
kegg_ids['Target'] = kegg_ids['Target'].str.replace('t1$', '', regex=True)
#rename column
kegg_ids_renamed = kegg_ids.rename(columns={'Target':'gene'})
#remove ID column in read counts
read_counts_cleaned = read_counts.iloc[:,1:]
#rename columns so they are treatment conditions
new_columns = read_counts_cleaned.iloc[1].tolist()
# Step 2: Assign the new column names to the DataFrame (starting from the second row)
read_counts_cleaned.columns = new_columns
#merge read_counts with KEGG IDs
kegg_counts_merged = pd.merge(kegg_ids_renamed,read_counts_cleaned,on='gene')
#create list of DEGs from gestalt
DEG_list = filtered_gestalt['KO'].tolist()
#filter read_counts to only DEGs
counts_filtered = kegg_counts_merged.loc[kegg_counts_merged['KEGG'].isin(DEG_list)]
#remove daphnia Gene IDs
counts_cleaned = counts_filtered.iloc[:,1:]
#save as csv
counts_cleaned.to_csv('counts_WGCNA.csv')
kegg_counts_merged.head()
filtered_gestalt.head()
counts_cleaned.head()

,KEGG,C01_1H,C02_1H,C03_1H,C04_1H,C05_1H,C06_1H,L01_1H,L02_1H,L03_1H,...,L02_7D,L03_7D,L05_7D,L06_7D,H01_7D,H02_7D,H03_7D,H04_7D,H05_7D,H06_7D
8,K01047,6.06751,5.858068,4.995574,5.112081,6.404495,6.131441,5.51084,6.09969,5.872585,...,2.449081,5.01446,3.203052,4.494388,3.597182,4.301904,5.320276,3.553359,4.928428,4.54695
300,K00910,4.421002,5.285725,3.711352,1.041112,4.553727,4.935187,5.238639,4.698746,4.665497,...,4.383841,4.876651,3.900979,4.747648,5.111019,4.500092,5.929753,4.102452,5.462572,4.972608
379,K01672,5.63052,3.884293,3.208083,5.371681,3.467853,4.734999,3.818224,3.790471,3.957132,...,5.245358,5.524868,6.412868,5.904011,5.543274,5.226795,6.246959,5.131253,5.776148,5.951237
410,K01672,7.855779,7.008721,8.804418,7.356212,7.51169,7.850716,10.266267,8.77806,9.636646,...,3.614957,3.613339,3.097102,2.6571,2.855075,1.639913,2.690415,5.839845,4.452585,2.914977
508,K01077,8.468334,8.157712,3.208083,8.495895,1.041112,7.96328,5.836434,8.157474,6.608527,...,8.638517,8.930154,8.543368,9.036829,8.235686,8.222226,8.952303,8.674834,8.674233,8.161136


In [30]:
#create metadata file
# Initialize lists to store treatment and timepoint data
treatment = []
timepoint = []

column_list = counts_cleaned.columns[1:].tolist()
#Iterate through the column names and extract treatment and timepoint
for col in column_list:
    # Determine treatment based on the first character
    if col.startswith('C'):
        treatment.append('control')
    elif col.startswith('L'):
        treatment.append('low')
    elif col.startswith('H'):
        treatment.append('high')
    
    # Extract timepoint from the column name (everything after the underscore)
    timepoint_value = col.split('_')[1]  # Get the part after the underscore
    timepoint.append(timepoint_value)

#Create a DataFrame for the metadata
metadata = pd.DataFrame({
    'SampleID': column_list,  # Column names as SampleID
    'Treatment': treatment,
    'TimePoint': timepoint
})
#save to csv

metadata.to_csv('metadata_WGCNA.csv')